---
layout: post  
title: Indexing Kmers  
date: 2020-12-05  
author: Cameron Prybol  

---

- Assume that you have a known kmer length, k, and a known alphabet (either DNA nucleotides ACGT or Amino Acids)

There are serveral ways that we can store kmers.
The simplest way would be to just store the actual kmer.
One of the most convenient ways to store kmers is to have a hash, which enables quick and constant time lookups to see if a given kmer exists.
The hash method doesn't scale well though since it requires every kmer to remain in memory.
Keeping all kmers in memory is often do-able, but since I'd like to only write one method that works everywhere regardless of available memory resources (we are working under the assumption that there is sufficient disk storage to store and analyze the data), we'll focus on methods that allow memory-mapping to disk such that we can work with datasets that are larger than available RAM, but will remain entirely in RAM when they can.

When memory-mapping to disk, we must use fixed-size datatypes.
The two options that seem most practical are:
- immutable, fixed-length containers of nucleotides or amino acids
    - Tuples
    - [Static Arrays](https://github.com/JuliaArrays/StaticArrays.jl)
- integers
    - for a given k-length, there is a finite # of possible kmers
    - for a given biological alphabet of N characters, the value is N^k
    - given an ordering of these characters, we can deterministically solve for the index that a given kmer would occupy in a sorted list of all possible kmers of k-length
    - thus, we can unambiguously map between an integer index and a given kmer with a known alphabet
    - this only becomes an issue when the size of the possible kmers is larger than what can be stored in native integer datatypes
        - e.g. UInt32, 64
        
Since I also want to keep track of the number of occurances of each kmer in a given dataset, I THINK that the most concise way to keep track of kmers is a sparse count-vector of the # of times each kmer was observed, where:
- size of the sparse count vector = N^k where N = size of alphabet and k = length of kmer
- kmers are mapped to an integer, such that the i-th index of the count vector represents the i-th kmer in a hypothetical dense sorted list of all possible kmers
- any kmer with counts > 0 exists, and thus the counts vector is sufficient to store the kmers and their frequencies

In [1]:
import Pkg
pkgs = [
    "BenchmarkTools",
    "BioSequences",
    "BioSymbols",
    "DataFrames",
    "GLM",
    "Primes",
    "ProgressMeter",
    "StaticArrays",
    "Statistics",
    "StatsPlots",
    "Test"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

Pkg.add("PlotlyJS")
StatsPlots.plotlyjs()

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-7868345806734811546\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2424009860299011010\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-7868345806734811546\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
┌ Warning: Error requiring PlotlyJS from Plots:
│ LoadError: LoadError: LoadError: UndefVarError: WebIO not defined
│ Stacktrace:
│  [1] top-level scope
│  [2] #macroexpand#36 at ./expr.jl:108 [inlined]
│  [3] macroexpand(::Module, ::Any) at ./expr.jl:107
│  [4] @require(::LineNumberNode, ::Module, ::Any, ::Any) at /home/jupyter-cjprybol/.julia/packages/Requires/Odp8W/src/require.jl:90
│  [5] include(::Function, ::Module, ::String) at ./Base.jl:380
│  [6] include at ./Base.jl:368 [inlined]
│  [7] include(::String) at /home/jupyter-cjprybol/.julia/packages/Plots/nHJea/src/Plots.jl:1
│  [8] top-level scope at /home/jupyter-cjprybol/.julia/packages/Plots/nHJea/src/init.jl:61
│  [9] eval at ./boot.jl:331 [inlined]
│  [10] eval at /home/jupyter-cjprybol/.julia/packages/Plots/nHJea/src/Plots.jl:1 [inlined]
│  [11] (::Plots.var"#323#356")() at /home/jupyter-cjprybol/.julia/packages

Plots.PlotlyJSBackend()

## Define the functions

In [2]:
"""
    index_to_kmer(index, k, alphabet)

Given an index equivalent to where a kmer would appear in a sorted list of all possible kmers
given the parameters k and the biological alphabet, return the kmer that exists in that index
"""
function index_to_kmer(index, K::Val{k}, alphabet) where k
    @assert k > 0 "invalid k: $k"
    N = length(alphabet)
    max_index = length(alphabet)^k
    @assert 0 < index <= max_index "invalid index: $index not within 1:$(max_index)"
    kmer = Vector{eltype(alphabet)}(undef, k)
    for i in k:-1:1
        divisor = N^(i-1)
        alphabet_index = Int(ceil(index/divisor))
        index = index % divisor
        if alphabet_index == 0
            alphabet_index = N
        end
        kmer[k-i+1] = alphabet[alphabet_index]
    end
    # https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-value-type
    return ntuple(i-> kmer[i], K)
end

index_to_kmer

In [3]:
"""
    kmer_to_index(kmer, alphabet)

Given a kmer and an alphabet of all biological symbols under consideration,
return the index that kmer would occupy in the sorted list of all possible kmers
"""
function kmer_to_index(kmer, alphabet)
    index = 0
    k = length(kmer)
    for i in k:-1:2
        alphabet_index = Int(findfirst(x -> x == kmer[k-i+1], alphabet))
        index += (alphabet_index - 1) * length(alphabet)^(i-1)
    end
    index += Int(findfirst(x -> x == kmer[end], alphabet))
    return index
end

kmer_to_index

## Here we will determine the largest possbile kmers that can be stored for a given alphabet and k-length

In [4]:
DNA_ALPHABET = filter(symbol -> BioSymbols.iscertain(symbol), BioSymbols.alphabet(BioSymbols.DNA))
AA_ALPHABET = filter(symbol -> BioSymbols.iscertain(symbol) && !BioSymbols.isterm(symbol), BioSymbols.alphabet(BioSymbols.AminoAcid))

(AA_A, AA_R, AA_N, AA_D, AA_C, AA_Q, AA_E, AA_G, AA_H, AA_I, AA_L, AA_K, AA_M, AA_F, AA_P, AA_S, AA_T, AA_W, AA_Y, AA_V, AA_O, AA_U)

In [5]:
for ALPHABET in (DNA_ALPHABET, AA_ALPHABET)
    println(eltype(ALPHABET))
    for T in (Int32, Int64, Int128)
        k = 0
        println(T)
        while BigInt(length(ALPHABET))^k < typemax(T)
            k += 1
        end
        println(k)
    end
    println()
end

BioSymbols.DNA
Int32
16
Int64
32
Int128
64

BioSymbols.AminoAcid
Int32
7
Int64
15
Int128
29



## Test that they are correct

In [6]:
Test.@testset "Test kmer <=> index transformations" begin
    for ALPHABET in (DNA_ALPHABET, AA_ALPHABET)
        for k in 1:3
            for (index, kmer) in enumerate(sort!(vec(collect(Iterators.product([ALPHABET for i in 1:k]...)))))
                Test.@test index == kmer_to_index(kmer, ALPHABET)
                Test.@test kmer == index_to_kmer(index, Val(k), ALPHABET)
            end
        end
    end
end

Test Summary:                       |  Pass  Total
Test kmer <=> index transformations | 22476  22476


Test.DefaultTestSet("Test kmer <=> index transformations", Any[], 22476, false)

## Assess for type instability

In [7]:
@code_warntype kmer_to_index(Tuple(BioSequences.randdnaseq(3)), DNA_ALPHABET)

Variables
  #self#::Core.Compiler.Const(kmer_to_index, false)
  kmer::Tuple{BioSymbols.DNA,BioSymbols.DNA,BioSymbols.DNA}
  alphabet::NTuple{4,BioSymbols.DNA}
  #4::var"#4#6"{Tuple{BioSymbols.DNA,BioSymbols.DNA,BioSymbols.DNA}}
  k::Int64
  @_6::Union{Nothing, Tuple{Int64,Int64}}
  index::Int64
  i::Int64
  #3::var"#3#5"{Tuple{BioSymbols.DNA,BioSymbols.DNA,BioSymbols.DNA},Int64,Int64}
  alphabet_index::Int64

Body::Int64
1 ─       Core.NewvarNode(:(#4))
│         (index = 0)
│         (k = Main.length(kmer))
│   %4  = (k::Core.Compiler.Const(3, false):-1:2)::Core.Compiler.Const(3:-1:2, false)
│         (@_6 = Base.iterate(%4))
│   %6  = (@_6::Core.Compiler.Const((3, 3), false) === nothing)::Core.Compiler.Const(false, false)
│   %7  = Base.not_int(%6)::Core.Compiler.Const(true, false)
└──       goto #4 if not %7
2 ┄ %9  = @_6::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%9, 1))
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = Main.:(var"#3#5")::Core.Compiler.Const

In [8]:
k = 3
@code_warntype index_to_kmer(rand(1:length(DNA_ALPHABET)^k), Val(k), DNA_ALPHABET)

Variables
  #self#::Core.Compiler.Const(index_to_kmer, false)
  index@_2::Int64
  K::Core.Compiler.Const(Val{3}(), false)
  alphabet::NTuple{4,BioSymbols.DNA}
  #1::var"#1#2"{Array{BioSymbols.DNA,1}}
  N::Int64
  max_index::Int64
  kmer::Array{BioSymbols.DNA,1}
  @_9::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  divisor::Int64
  alphabet_index::Int64
  index@_13::Int64
  @_14::Bool

Body::Tuple{BioSymbols.DNA,BioSymbols.DNA,BioSymbols.DNA}
1 ──       (index@_13 = index@_2)
│          Core.NewvarNode(:(#1))
│          Core.NewvarNode(:(N))
│          Core.NewvarNode(:(max_index))
│          Core.NewvarNode(:(kmer))
│          Core.NewvarNode(:(@_9))
│    %7  = ($(Expr(:static_parameter, 1)) > 0)::Core.Compiler.Const(true, false)
│          %7
└───       goto #3
2 ──       Core.Compiler.Const(:(Base.getproperty(Base.Main, :Base)), false)
│          Core.Compiler.Const(:(Base.getproperty(%10, :string)), false)
│          Core.Compiler.Const(:(Base.string("invalid k: ", $(Expr(:static_p

## Kmer to Index

Observe that memory allocations and gc time are zero, while the indexing algorithm scales linearly with the size of k.
Linear scaling to size of k while the # of possible kmers increases exponentially means that this approach should scale better than 

In [17]:
ks = []
results = []
ProgressMeter.@showprogress for k in Primes.primes(3, 32)
    kmer = Tuple(BioSequences.randdnaseq(k))
    result = BenchmarkTools.@benchmark kmer_to_index($kmer, $DNA_ALPHABET)
    push!(ks, k)
    push!(results, result)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:29


assert that allocations and memory usage are zero

In [18]:
first(results)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     37.932 ns (0.00% GC)
  median time:      37.933 ns (0.00% GC)
  mean time:        38.424 ns (0.00% GC)
  maximum time:     328.004 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     992

In [19]:
Test.@testset "Test kmer <=> index transformations allocations" begin
    for result in results
        Test.@test result.allocs == 0
        Test.@test all(x -> x == 0, result.gctimes)
    end
end

Test Summary:                                   | Pass  Total
Test kmer <=> index transformations allocations |   20     20


Test.DefaultTestSet("Test kmer <=> index transformations allocations", Any[], 20, false)

In [20]:
xs = Float64[]
ys = Float64[]
subsampling_size = 10
for (k, result) in zip(ks, results)
    for time in rand(result.times, subsampling_size)
        push!(xs, k)
        push!(ys, time)
    end
end

p = StatsPlots.scatter(
    xs,
    ys,
    title = "kmer -> index conversion for DNA alphabet",
    ylabel="median nano-seconds per lookup",
    xlabel="k length",
    label="benchmark results",
    legend=:outertopright
)

linear_model = GLM.lm(GLM.@formula(y ~ x), DataFrames.DataFrame(x = xs, y = ys))
fit_xs = collect(minimum(xs):maximum(xs))
fit_ys = map(x -> GLM.coef(linear_model)[1] + GLM.coef(linear_model)[2] * x, fit_xs)

StatsPlots.plot!(p, 
    fit_xs,
    fit_ys,
    label="fit trendline"
)

<!DOCTYPE html>
 
 
 Plots.jl

## Index to Kmer

In [21]:
ks = []
results = []
ProgressMeter.@showprogress for k in Primes.primes(3, 32)
    index = rand(1:length(DNA_ALPHABET)^k)
    result = BenchmarkTools.@benchmark index_to_kmer($index, $(Val(k)), $DNA_ALPHABET)
    push!(ks, k)
    push!(results, result)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:32


In [22]:
first(results)

BenchmarkTools.Trial: 
  memory estimate:  96 bytes
  allocs estimate:  1
  --------------
  minimum time:     109.483 ns (0.00% GC)
  median time:      111.470 ns (0.00% GC)
  mean time:        128.347 ns (10.70% GC)
  maximum time:     10.419 μs (98.80% GC)
  --------------
  samples:          10000
  evals/sample:     929

In [23]:
xs = Float64[]
ys = Float64[]
subsampling_size = 10
for (k, result) in zip(ks, results)
    for time in rand(result.times, subsampling_size)
        push!(xs, k)
        push!(ys, time)
    end
end

p = StatsPlots.scatter(
    xs,
    ys,
    title = "index -> kmer conversion for DNA alphabet",
    ylabel="median nano-seconds per conversion",
    xlabel="k length",
    label="benchmark results",
    legend=:outertopright
)

linear_model = GLM.lm(GLM.@formula(y ~ x), DataFrames.DataFrame(x = xs, y = ys))
fit_xs = collect(minimum(xs):maximum(xs))
fit_ys = map(x -> GLM.coef(linear_model)[1] + GLM.coef(linear_model)[2] * x, fit_xs)

StatsPlots.plot!(p, 
    fit_xs,
    fit_ys,
    label="fit trendline"
)

<!DOCTYPE html>
 
 
 Plots.jl